In [57]:
import requests

url = "https://realtor.p.rapidapi.com/properties/list-for-rent"

querystring = {"state_code":"NY","city":"New York City","limit":"200","offset":"0","sort":"relevance"}

headers = {
    "X-RapidAPI-Key": "7c62f71be8msh61ccb7db7073854p1a9640jsnd13a2fbfc675",
    "X-RapidAPI-Host": "realtor.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

In [58]:
columns=['prop_type',
         'address_new',
         'price',
         'beds',
         'baths',
         'sqft',
         'rank',
         'source']

In [59]:
import numpy as np
import pandas as pd
data = response.json()
df = pd.DataFrame.from_dict(data['listings'])
df = df[columns]

In [60]:
pd.set_option('display.max_columns',50)

In [61]:
df.head()

,prop_type,address_new,price,beds,baths,sqft,rank,source
0,condop,"{'city': 'Staten Island', 'country': 'USA', 'c...",$1/mo,1,2,sq ft N/A,1,mls
1,condop,"{'city': 'Staten Island', 'country': 'USA', 'c...",$1/mo,1,2,sq ft N/A,2,mls
2,condop,"{'city': 'Staten Island', 'country': 'USA', 'c...",$1/mo,1,2,sq ft N/A,3,mls
3,apartment,"{'city': 'New York', 'country': 'USA', 'county...",$1+/mo,S-1,1-2,439+ sq ft,4,community
4,apartment,"{'city': 'New York', 'country': 'USA', 'county...",$1/mo,1,1,1 sq ft,5,community


In [62]:
df['prop_type'].value_counts()

condo             120
single_family      40
apartment          27
condop              5
townhome            3
multi_family        3
other               1
duplex_triplex      1
Name: prop_type, dtype: int64

In [63]:
import re
df['price'] = df['price'].apply(lambda x:re.sub("[^0-9]","",x))

#### Price = 1 is definitely a typo or a ridiculous mistake

In [64]:
df = df[df['price']!='1']

In [65]:
df['sqft'].value_counts()

sq ft N/A      121
650 sq ft        6
120 sq ft        4
N/A sq ft        4
107 sq ft        3
100 sq ft        3
500 sq ft        2
800 sq ft        2
102 sq ft        2
200 sq ft        2
114 sq ft        2
113 sq ft        2
112 sq ft        2
2,296 sq ft      1
103 sq ft        1
101 sq ft        1
110 sq ft        1
414+ sq ft       1
121 sq ft        1
438+ sq ft       1
850 sq ft        1
132 sq ft        1
140 sq ft        1
1,936 sq ft      1
135 sq ft        1
153 sq ft        1
889 sq ft        1
208 sq ft        1
680 sq ft        1
106 sq ft        1
1,521 sq ft      1
136 sq ft        1
550+ sq ft       1
179 sq ft        1
128 sq ft        1
104 sq ft        1
900 sq ft        1
600 sq ft        1
126 sq ft        1
175 sq ft        1
155 sq ft        1
700 sq ft        1
145 sq ft        1
133 sq ft        1
108 sq ft        1
111 sq ft        1
178 sq ft        1
122 sq ft        1
315 sq ft        1
123 sq ft        1
Name: sqft, dtype: int64

In [66]:
def na(x):
    if "N/A" in x: 
        return np.nan
    else:
        return re.sub('[^0-9]','',x)

In [67]:
df['sqft'] = df['sqft'].apply(na)

In [68]:
df['sqft'] = df['sqft'].astype(float)

In [69]:
df.isnull().sum()

prop_type        0
address_new      0
price            0
beds             0
baths            0
sqft           125
rank             0
source           0
dtype: int64

#### SQFT has 125 missing values

In [70]:
df['sqft'].median()

136.0

In [17]:
from sklearn.impute import KNNImputer
impute_knn = KNNImputer(n_neighbors=2)
impute_knn.fit_transform(df)

ValueError: could not convert string to float: 'townhome'

In [71]:
df.reset_index(drop=True,inplace=True)

In [73]:
df['city']=None
# df['Country']=None
# df['State']=None
for i in range(0,len(df)):
    temp = df['address_new'][i]
    df['city'][i] = temp['city']
#     df['Country'][i] = temp['country']
#     df['State'][i] = temp['state']

C:\Users\Shubh\AppData\Local\Temp\ipykernel_6380\1675835345.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['city'][i] = temp['city']


In [74]:
df

,prop_type,address_new,price,beds,baths,sqft,rank,source,city
0,townhome,"{'city': 'Staten Island', 'country': 'USA', 'c...",101,2,3,121.0,9,mls,Staten Island
1,single_family,"{'city': 'Far Rockaway', 'country': 'USA', 'co...",150,S,Call,NaN,10,mls,Far Rockaway
2,single_family,"{'city': 'Brooklyn', 'country': 'USA', 'county...",800,1,1,NaN,11,unit_rental,Brooklyn
3,single_family,"{'city': 'South Ozone Park', 'country': 'USA',...",800,S,1,NaN,12,mls,South Ozone Park
4,townhome,"{'city': 'New York', 'country': 'USA', 'county...",850,1,1,179.0,13,unit_rental,New York
...,...,...,...,...,...,...,...,...,...
187,condo,"{'city': 'New York', 'country': 'USA', 'county...",1400,5,2,NaN,24,unit_rental,New York
188,condo,"{'city': 'New York', 'country': 'USA', 'county...",1400,4,2,123.0,25,unit_rental,New York
189,condo,"{'city': 'New York', 'country': 'USA', 'county...",1400,4,2,NaN,26,unit_rental,New York
190,condo,"{'city': 'New York', 'country': 'USA', 'county...",1400,4,2,NaN,27,unit_rental,New York


In [75]:
df.drop(columns='address_new',inplace=True)
df

,prop_type,price,beds,baths,sqft,rank,source,city
0,townhome,101,2,3,121.0,9,mls,Staten Island
1,single_family,150,S,Call,NaN,10,mls,Far Rockaway
2,single_family,800,1,1,NaN,11,unit_rental,Brooklyn
3,single_family,800,S,1,NaN,12,mls,South Ozone Park
4,townhome,850,1,1,179.0,13,unit_rental,New York
...,...,...,...,...,...,...,...,...
187,condo,1400,5,2,NaN,24,unit_rental,New York
188,condo,1400,4,2,123.0,25,unit_rental,New York
189,condo,1400,4,2,NaN,26,unit_rental,New York
190,condo,1400,4,2,NaN,27,unit_rental,New York


In [76]:
df.beds.value_counts()

4      79
1      33
3      23
S      18
6      16
5      14
2       6
S-2     2
S-3     1
Name: beds, dtype: int64

In [77]:
dele = df[(df['baths']=='0-2') | (df['baths']=='1-2') | (df['baths']=='Call') | (df['beds']=='S') | (df['beds']=='S-2') | (df['beds']=='S-3')].index

In [78]:
len(dele)

21

In [79]:
df.drop(dele,inplace=True)

In [80]:
df.reset_index(drop=True,inplace=True)

In [81]:
df

,prop_type,price,beds,baths,sqft,rank,source,city
0,townhome,101,2,3,121.0,9,mls,Staten Island
1,single_family,800,1,1,NaN,11,unit_rental,Brooklyn
2,townhome,850,1,1,179.0,13,unit_rental,New York
3,condo,850,4,1,128.0,14,unit_rental,New York
4,condo,875,4,2,104.0,15,unit_rental,New York
...,...,...,...,...,...,...,...,...
166,condo,1400,5,2,NaN,24,unit_rental,New York
167,condo,1400,4,2,123.0,25,unit_rental,New York
168,condo,1400,4,2,NaN,26,unit_rental,New York
169,condo,1400,4,2,NaN,27,unit_rental,New York


In [82]:
df.to_csv('WithNanValues.csv')

In [83]:
data = df.drop(columns = 'rank')

In [84]:
data

,prop_type,price,beds,baths,sqft,source,city
0,townhome,101,2,3,121.0,mls,Staten Island
1,single_family,800,1,1,NaN,unit_rental,Brooklyn
2,townhome,850,1,1,179.0,unit_rental,New York
3,condo,850,4,1,128.0,unit_rental,New York
4,condo,875,4,2,104.0,unit_rental,New York
...,...,...,...,...,...,...,...
166,condo,1400,5,2,NaN,unit_rental,New York
167,condo,1400,4,2,123.0,unit_rental,New York
168,condo,1400,4,2,NaN,unit_rental,New York
169,condo,1400,4,2,NaN,unit_rental,New York


In [32]:
target_variable = 'sqft'
independent_variables = ['prop_type', 'price', 'beds', 'baths', 'source', 'City']

In [87]:
data.isnull().sum()

prop_type      0
price          0
beds           0
baths          0
sqft         115
source         0
city           0
dtype: int64

In [86]:
171-56

115

In [88]:
X_train = data[~data['sqft'].isnull()].drop(columns='sqft')
y_train = data[~data['sqft'].isnull()]['sqft']

In [91]:
y_train

0       121.0
2       179.0
3       128.0
4       104.0
6       900.0
9       113.0
13      600.0
19      200.0
23      100.0
24      102.0
25      126.0
27      107.0
31      107.0
33      175.0
34      155.0
36      800.0
39      145.0
40      133.0
44      108.0
51      111.0
56      178.0
65      122.0
75      136.0
76      140.0
86      106.0
87      110.0
88      103.0
89      800.0
93      100.0
95      112.0
96      114.0
97      114.0
101     650.0
103     500.0
107     101.0
111     120.0
112     120.0
118     120.0
126     650.0
129     113.0
132     850.0
140     107.0
141     132.0
142    1936.0
145     102.0
147     135.0
151     100.0
152     650.0
154     153.0
156     120.0
157     650.0
158     889.0
160     208.0
163     680.0
167     123.0
170     112.0
Name: sqft, dtype: float64

In [89]:
X_train

,prop_type,price,beds,baths,source,city
0,townhome,101,2,3,mls,Staten Island
2,townhome,850,1,1,unit_rental,New York
3,condo,850,4,1,unit_rental,New York
4,condo,875,4,2,unit_rental,New York
6,single_family,900,1,2,mls,Corona
9,condo,925,3,1,unit_rental,New York
13,single_family,950,1,1,mls,Jamaica
19,single_family,1000,1,1,unit_rental,New York
23,condo,1025,4,1,unit_rental,New York
24,condo,1050,4,1,unit_rental,New York


In [96]:
X_test = data[data['sqft'].isnull()].drop(columns='sqft')

In [97]:
X_test

,prop_type,price,beds,baths,source,city
1,single_family,800,1,1,unit_rental,Brooklyn
5,condo,875,4,1,unit_rental,New York
7,apartment,900,1,1,unit_rental,Bronx
8,single_family,900,1,1,unit_rental,Brooklyn
10,apartment,945,1,1,unit_rental,Brooklyn
...,...,...,...,...,...,...
164,apartment,1400,1,1,unit_rental,Queens
165,single_family,1400,4,2,unit_rental,New York
166,condo,1400,5,2,unit_rental,New York
168,condo,1400,4,2,unit_rental,New York


In [98]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [100]:
numeric_features = ['price', 'beds','baths']
categorical_features = ['prop_type', 'source','city']

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Create a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create a pipeline with the preprocessor and the linear regression model
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['price', 'beds', 'baths']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['prop_type', 'source',
                                                   'city'])])),
                ('regressor', LinearRegression())])

In [101]:
pipeline.predict(X_test)

array([1.11781043e+03, 3.03905764e+01, 5.76225781e+02, 1.15755237e+03,
       7.08174169e+02, 6.00000000e+02, 1.23186969e+00, 1.23186969e+00,
       1.23186969e+00, 7.01325173e+01, 6.01284611e+02, 6.00702736e+02,
       9.62196249e+02, 7.18741444e+02, 8.00680025e+01, 7.15683581e+01,
       6.29103970e+02, 6.57474584e+02, 1.09874458e+02, 6.41026552e+02,
       1.19809943e+02, 9.79356606e+02, 9.20391443e+02, 9.14393285e+01,
       1.22033876e+02, 8.92153958e+01, 1.31969361e+02, 1.31969361e+02,
       6.64871716e+02, 8.07157515e+01, 8.07157515e+01, 1.39680914e+02,
       7.27811073e+02, 7.31785267e+02, 9.06512367e+01, 9.06512367e+01,
       7.35759461e+02, 7.35759461e+02, 8.77432177e+02, 6.84742687e+02,
       1.59551884e+02, 1.00586722e+02, 1.00586722e+02, 1.59551884e+02,
       1.73371340e+02, 1.00586722e+02, 8.09516118e+02, 6.58122333e+02,
       6.88716881e+02, 6.92691075e+02, 1.69487369e+02, 1.10522207e+02,
       1.38892822e+02, 1.69487369e+02, 7.99276596e+01, 1.69487369e+02,
      

In [103]:
predicted_scores = pipeline.predict(X_test)

predicted_scores_original = np.round(predicted_scores)  # Rounding to nearest integer

# Print the original and predicted scores
for predicted in predicted_scores_original:
    print(f"Predicted Score (Original Form): {predicted}")

Predicted Score (Original Form): 1118.0
Predicted Score (Original Form): 30.0
Predicted Score (Original Form): 576.0
Predicted Score (Original Form): 1158.0
Predicted Score (Original Form): 708.0
Predicted Score (Original Form): 600.0
Predicted Score (Original Form): 1.0
Predicted Score (Original Form): 1.0
Predicted Score (Original Form): 1.0
Predicted Score (Original Form): 70.0
Predicted Score (Original Form): 601.0
Predicted Score (Original Form): 601.0
Predicted Score (Original Form): 962.0
Predicted Score (Original Form): 719.0
Predicted Score (Original Form): 80.0
Predicted Score (Original Form): 72.0
Predicted Score (Original Form): 629.0
Predicted Score (Original Form): 657.0
Predicted Score (Original Form): 110.0
Predicted Score (Original Form): 641.0
Predicted Score (Original Form): 120.0
Predicted Score (Original Form): 979.0
Predicted Score (Original Form): 920.0
Predicted Score (Original Form): 91.0
Predicted Score (Original Form): 122.0
Predicted Score (Original Form): 8

In [104]:
predicted_scores_original.shape

(115,)

In [108]:
X_test.index

Int64Index([  1,   5,   7,   8,  10,  11,  12,  14,  15,  16,
            ...
            153, 155, 159, 161, 162, 164, 165, 166, 168, 169],
           dtype='int64', length=115)